In [4]:
dates=[ '0501', '0502', '0503', '0504', '0505', '0506', '0507','0508', '0509', '0510', '0511', '0512', '0513', '0514', '0515','0516', '0517', '0518', '0519', '0520', '0521', '0522', '0523','0524', '0525', '0526', '0527', '0528', '0529', '0530', '0531', '0602', '0603', '0604', '0605', '0606', '0607', '0608','0609', '0610', '0611', '0612', '0613', '0614', '0615', '0616','0617', '0618', '0619', '0620', '0621', '0622', '0623', '0624','0625', '0626', '0627', '0628', '0629', '0630']

In [14]:
for index,date in enumerate(dates):
    if date=='0620':
        print(index)

49


In [13]:
print(dates[47])

0618


In [1]:
import numpy

In [15]:
total=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\60天统计\间隔_5min.npy',allow_pickle=True).item()

In [54]:
target_segs=[9, 24, 42, 256, 267, 268, 277]
train={}
test={}
validation={}
for seg in target_segs:
    train[seg]=total[seg][2:34]+total[seg][37:48]#5.2-6.3+6.7-6.17(真实日期)
    test[seg]=total[seg][49:]#6.19-6.29

In [55]:
for seg in target_segs:
    train_arr=[]
    for day in train[seg]:
        train_arr+=day
    test_arr=[]
    for day in test[seg]:
        test_arr+=day
    # max_train=numpy.max(train_arr)
    # min_train=numpy.min(train_arr) 
    # max_min_table[seg]=[max_train,min_train]
    max_train,min_train=max_min_table[seg]
    max_train_arr=numpy.array([max_train for i in range(7740)])
    min_train_arr=numpy.array([min_train for i in range(7740)])
    max_test_arr=[max_train for i in range(1980)]
    min_test_arr=numpy.array([min_train for i in range(1980)])
    
    bias_train=(max_train_arr-min_train_arr)
    bias_test=(max_test_arr-min_test_arr)
    # if numpy.sum(numpy.where(bias_test==0,1,0))>0:
        # print(seg)
    train_arr=numpy.around((train_arr-min_train_arr)/bias_train,2)
    test_arr=numpy.around((test_arr-min_test_arr)/bias_test,2)
    
    train[seg]=train_arr[:32*180]
    test[seg]=test_arr
    validation[seg]=train_arr[32*180:]

In [25]:
max_min_table

{9: [392.0, 69.0],
 24: [287.0, 29.0],
 42: [209.4, 34.0],
 256: [224.2, 31.0],
 267: [1550.0, 63.0],
 268: [348.8, 51.0],
 277: [165.0, 64.0]}

In [56]:
#训练集
train_seg={}
for seg,record in train.items():
    input_c=[]
    input_d=[]
    input_w=[]
    output=[]
    for day in range(4,32):
        valid=record[day*180:(day+1)*180] # a list
        for i in range(176):
            input_c.append(valid[i:i+4])
            output.append(valid[i+4])
            tmp_d=[record[(day-step)*180+i] for step in range(4)] #前四天
            # tmp_w=[record[(day-step*7)*180+i] for step in range(1)]#前一周
            input_d.append(tmp_d)
            # input_w.append(tmp_w)
    train_seg[seg]=[input_c,input_d,output]
    # train_seg[seg]=[input_c,input_d,input_w,output]


In [83]:
#测试集
test_seg={}
for seg,record in test.items():
    input_c=[]
    input_d=[]
    input_w=[]
    output=[]
    for day in range(4,11):
        valid=record[day*180:(day+1)*180] # a list
        for i in range(176):
            input_c.append(valid[i:i+4])
            output.append(valid[i+4])
            tmp_d=[record[(day-step)*180+i] for step in range(4)] #前四天
            # tmp_w=[record[(day-step*7)*180+i] for step in range(1)]#前一周
            input_d.append(tmp_d)
            # input_w.append(tmp_w)
    # test_seg[seg]=[input_c,input_d,input_w,output]
    test_seg[seg]=[input_c,input_d,output]

In [84]:
#验证集
validation_seg={}
for seg,record in validation.items():
    input_c=[]
    input_d=[]
    input_w=[]
    output=[]
    for day in range(4,11):
        valid=record[day*180:(day+1)*180] # a list
        for i in range(176):
            input_c.append(valid[i:i+4])
            output.append(valid[i+4])
            tmp_d=[record[(day-step)*180+i] for step in range(4)] #前四天
            # tmp_w=[record[(day-step*7)*180+i] for step in range(1)]#前一周
            input_d.append(tmp_d)
            # input_w.append(tmp_w)
    # test_seg[seg]=[input_c,input_d,input_w,output]
    validation_seg[seg]=[input_c,input_d,output]

In [85]:
len(test_seg[9][0])

1232

In [86]:
# numpy.save(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_1\train.npy',train_seg)
numpy.save(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_1\test.npy',test_seg)
numpy.save(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_1\validation.npy',validation_seg)

In [ ]:
time_stamp=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\40天原始记录\time_stamp.npy',allow_pickle=True).item()
ts_list=list(time_stamp.keys())

In [ ]:
ts=[]
for each in ts_list:
    tmp=each.split(':')
    ts.append(tmp[0]+tmp[1])

In [ ]:
ts[:5]

In [ ]:
new_ts=[]
for i in range(180):
    new_ts.append(ts[i*5])

In [ ]:
from keras.preprocessing.text import one_hot

In [ ]:
encoded_time_stamp=[]
for i in new_ts:
    encoded_time_stamp+=one_hot(i,180)

In [ ]:
numpy.save(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\encoded_time_stamp.npy',encoded_time_stamp)

In [2]:
dates_dict=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\60天统计\收集日期对应工作日.npy',allow_pickle=True).item()

In [76]:
day_of_week=['0612', '0613', '0614', '0615', '0616','0617', '0618']

In [77]:
len(day_of_week)

7

In [78]:
test_day_of_week=[]
for i in day_of_week:
    test_day_of_week+=[dates_dict[i] for j in range(176)]
    # test_day_of_week+=numpy.repeat([dates_dict[i]],176,axis=0)

In [79]:
numpy.save(r'C:\Users\e-wel\Documents\Python Scripts\DeepPF\log_1\validate_day_of_week.npy',test_day_of_week)